# 네이버 자동차 크롤링
## 문제1)

In [2]:
# ChromeDriver 자동 설치 모듈
import chromedriver_autoinstaller
# Chrome을 제어하기 위한 객체
from selenium import webdriver
# Chrome이 웹 페이지 로딩을 완료 할 때까지 최대 n초간 대기하는 기능.
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support.select import Select
from selenium.webdriver.common.alert import Alert
from selenium.webdriver.chrome.options import Options
from bs4 import BeautifulSoup
import pandas as pd
from pandas import DataFrame
# 파이썬 프로그램에 지정된 시간동안 랙을 거는 기능을 위해 사용
import time
#경우의 수
import itertools
import openpyxl
import numpy as np

In [3]:
# 크롬드라이버 자동 설치
chromedriver_autoinstaller.install()

'c:\\users\\junhong\\appdata\\local\\programs\\python\\python38\\lib\\site-packages\\chromedriver_autoinstaller\\91\\chromedriver.exe'

In [4]:
df = DataFrame()

In [7]:
# 크롬드라이버를 통해 크롬을 실행시킴
# -> driver 객체는 Chrome 자체

driver = webdriver.Chrome()
# 크롬브라우저가 준비될 때 까지 최대 5초씩 대기

driver.implicitly_wait(5)

driver.get("https://auto.naver.com/car/mainList.nhn")
    # 페이지를 이동하는 동안 최대 2초간 대기
time.sleep(2)

for count in range(0,1):
    while(True):
        html = driver.page_source
        soup = BeautifulSoup(html, "html.parser")

        car_list = soup.select('#content > div.model_group_new > ul > li')
        list_len = len(car_list) + 1


        for i in range(1, list_len):
            try:
                car = WebDriverWait(driver, 3).until(lambda x: x.find_element_by_css_selector("#content > div.model_group_new > ul > li:nth-child(%s) > div > div > span > a" %i))
                car.click()
                time.sleep(1)
            except:
                break
            html = driver.page_source
            soup = BeautifulSoup(html, "html.parser")

            name_sel = soup.select('#container > div.spot_end.new_end > div.info_group > div.end_model > h3')
            name = name_sel[0].text.strip()

            try:
                price_sel = soup.select('#container > div.spot_end.new_end > div.info_group > div.sale > p > span.cont')
                price = price_sel[0].text.strip()
                won_check = soup.select('#container > div.spot_end.new_end > div.info_group > div.sale > p > span:nth-of-type(3)')
                is_won = won_check[0].text.strip()

                if is_won != '만원':
                    driver.back()
                    continue
            except:
                price_none = soup.select('#container > div.spot_end.new_end > div.info_group > div.sale.none > p')
                if price_none:
                    driver.back()
                    continue

            info = WebDriverWait(driver, 3).until(lambda x: x.find_element_by_css_selector("#content > div.tab_mnu.v1 > ul > li.data > a"))
            info.click()

            time.sleep(1)

            html = driver.page_source
            soup = BeautifulSoup(html, "html.parser")

            columns_data = []

            for i in range(1,6):
                tmp = soup.select(".lineup_btm_th > div:nth-of-type(%s) > ul" %i)
                columns_data.append(tmp[0].text.strip())
            
            columns = []
            for i in columns_data:
                tmp = i.split('\n')
                for j in tmp:
                    if j.find(' ') > 0:
                        columns.append(j[:j.find(' ')])
                    else:
                        columns.append(j)
           
            columns = np.array(columns)
            new_columns = np.delete(columns, [0,2,3,4,5,6,7])
            new_columns = np.append(new_columns, np.array(['차명', '가격']))
            for i,v in enumerate(new_columns):
                if v == '연비(등급)\xa0':
                    new_columns[i] = '연비(등급)'

            data = None
            all_model = soup.select(".col_contents .lineup_btm_td")
            for i in all_model:
                tmp = i.select('li')
                for j in tmp:
                    if len(j.text.strip()) >0:
                        data=tmp
                        break
            print(data)
            for i,v in enumerate(data):
                data[i] = v.text.strip().replace(' ', '').replace('\n', ' ')

            data = np.array(data)
            new_data = np.delete(data, [0,2,3,4,5,6,7])
            new_data = np.append(new_data, np.array([name, price[price.find('~') + 1 :]]))

            df = df.append(pd.Series(new_data,index=new_columns),ignore_index=True)

            driver.back()
            driver.back()

        try:
            next_page = WebDriverWait(driver, 3).until(lambda x: x.find_element_by_css_selector("#content > div.paginate2 > a.next"))
            next_page.click()
        except:
            break
    glob_car = WebDriverWait(driver, 3).until(lambda x: x.find_element_by_css_selector('#lnb > ul > li.car.selected > div > ul > li.glob > a'))
    glob_car.click()

[<li class="_line_0">I4</li>, <li class="_line_1">싱글 터보</li>, <li class="_line_2">1,998cc</li>, <li class="_line_3">
                                        
                                        
                                            
                                            디젤
                                        
                                    </li>, <li class="_line_4" style="height: 46px;">
                                        
                                            
                                            자동 12.8~13.1 km/ℓ
                                             (3등급)
                                            
                                        
                                        
                                    </li>, <li class="_line_5">5인승</li>, <li class="_line_6">풀타임 4륜구동</li>, <li class="_line_7">
                                        
                                            자동 8단
                                    

NoSuchWindowException: Message: no such window: target window already closed
from unknown error: web view not found
  (Session info: chrome=91.0.4472.164)


In [236]:
index_df = df.set_index('차명')

In [242]:
index_df

,CO²배출량,가격,가속성능(0-100,공차중량,과급방식,구동방식,배기량,변속기,스티어링,승차인원,...,후륜제동장치,후륜타이어,합산출력,합산토크,1회,모터최대출력,모터최대토크,에너지용량,수소연료탱크용량,항속거리
차명,,,,,,,,,,,,,,,,,,,,,
2021 아반떼 N,158.0~164.0g/km,"3,402",정보없음,"1,450~1,485kg",싱글터보,전륜구동,"1,998cc","수동6단 , 자동(DCT)8단",랙앤피니언,5인승,...,V디스크,245mm/35/19inch,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2022 스포티지,146.0~150.0g/km,"3,731",정보없음,"1,685~1,715kg",싱글터보,풀타임4륜구동,"1,998cc",자동8단,랙앤피니언,5인승,...,디스크,235mm/55~65/17~19inch,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2022 쏘렌토,139.0~147.0g/km,"4,273",정보없음,"1,820~1,835kg",싱글터보,풀타임4륜구동,"2,151cc",자동(DCT)8단,랙앤피니언,5인승,...,디스크,235~255mm/45~60/18~20inch,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2020 아반떼,130.0~136.0g/km,"2,779",정보없음,"1,310~1,340kg",싱글터보,전륜구동,"1,598cc","수동6단 , 자동(DCT)7단",랙앤피니언,5인승,...,디스크,225mm/40/18inch,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2022 베뉴,121.0~125.0g/km,"2,236",정보없음,"1,180~1,215kg",자연흡기,전륜구동,"1,598cc",자동(CVT),랙앤피니언,5인승,...,디스크,185~205mm/55~65/15~17inch,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2019 토요타 시에나,208.0g/km,"5,800",정보없음,"2,175kg",자연흡기,풀타임4륜구동,"3,456cc",자동8단,랙앤피니언,7인승,...,디스크,235mm/55/18inch,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2019 동풍소콘 C31,정보없음,"1,250",정보없음,"1,215kg",자연흡기,후륜구동,"1,491cc",수동5단,정보없음,2인승,...,드럼,185mm/80/14inch,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2020 미니 클럽맨 SD,142.0g/km,"4,900",7.7초,"1,595kg",싱글터보,전륜구동,"1,995cc",자동8단,랙앤피니언,5인승,...,디스크,225mm/40/18inch,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [243]:
index_df.to_excel('자동차정보.xlsx',index=True)